<a href="https://colab.research.google.com/github/AhmedQureshi89/Battot/blob/main/Tr_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.4 MB/s eta 0:00:00


In [69]:
!pip install transformers

In [70]:
import os
import groq
!pip install transformers torch

from transformers import AutoTokenizer, AutoModelForCausalLM


In [71]:
from google.colab import userdata
os.environ["GROQ_API_KEY"] = "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"

In [72]:
client = groq.Groq()

In [84]:
llama3_bB_8192 = "llama3-70b-8192"
llama3_1_70B_versatile = "llama3.1_70B_versatile"



In [85]:
def generate_text(prompt, max_tokens=100):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model= llama3_bB_8192,
        max_tokens=max_tokens,
    )
    return chat_completion.choices[0].message.content

In [100]:
def translation_agent(text, source_lang, target_lang, temperature=0.7):
    system_prompt = """You are a multilingual translation agent. Your task is to:
    1. Identify the language of the input text if not specified.
    2. Translate the text to the target language.
    3. Provide a brief explanation of any idiomatic expressions or cultural nuances in the translation.
    """

    user_prompt = f"""Source language: {source_lang if source_lang else 'Unknown (please identify)'}
    Target language: {target_lang}
    Text to translate: {text}

    Please provide:
    1. The translation
    2. A brief explanation of any interesting translation choices or cultural nuances
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model= llama3_bB_8192,
        max_tokens=1000,
        temperature=temperature
    )
    return chat_completion.choices[0].message.content.strip()

In [101]:
# Test the translation agent with different parameters
text_to_translate = "Break a leg!"
source_lang = "English"
target_lang = "عربي"

# More focused translation
focused_result = translation_agent(text_to_translate, source_lang, target_lang, temperature=0.3)
print("Focused Translation:")
print(focused_result)
print("\n" + "="*50 + "\n")

# More creative translation
creative_result = translation_agent(text_to_translate, source_lang, target_lang, temperature=0.9)
print("Creative Translation:")
print(creative_result)

Focused Translation:
Here's the translation:

**Translation:** كلّ الحظ السعيد لك! (Kul al-ḥaẓ al-saʿīd lak!)

**Explanation:** The idiomatic expression "Break a leg!" is a common English phrase used to wish someone good luck, especially before a performance or a challenging situation. In Arabic, the literal translation of "Break a leg!" wouldn't make sense, so I opted for a more culturally equivalent phrase that conveys the same sentiment.

The translation "كلّ الحظ السعيد لك!" (Kul al-ḥaẓ al-saʿīd lak!) is a common Arabic expression that means "All the best luck to you!" or "Good luck to you!" The phrase is often used in similar contexts, such as before an exam, a job interview, or a performance.

In Arabic culture, it's customary to express good wishes and blessings to others, especially in situations where they might need an extra boost of confidence or luck. This translation choice aims to preserve the spirit of the original phrase while adapting it to the cultural nuances of the 

In [98]:
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=max_length, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def translate_text(text,source_lang, target_language):

    translated_text = translation_agent(text, source_lang, target_lang, temperature=0.3)
    return translated_text

def self_assess_translation(original_text, translated_text, target_language):


    system_prompt = """You are translation quality controll agent. Your task is to:
    1. Assess the translation and identify any potential doubts or areas of uncertainty.
    """

    user_prompt = f"""original_text: '{original_text}'\n"
    Translated Text: '{translated_text}'\n"
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model= llama3_bB_8192,
        max_tokens=1000,
        # temperature=temperature  # Remove this line as 'temperature' is not defined in this function
    )
    return chat_completion.choices[0].message.content.strip()
    #    return self_assessment  # Remove this line as it's unreachable and 'self_assessment' is not defined


def resolve_self_assessment(original_text, translated_text, self_assessment):
    system_prompt = """You are translation Self-Assessment resovler agent. Your task is to:
    1. Rsolve the Self-Assessment.
    """

    user_prompt = f"""original_text: '{original_text}'\n"
    Translated Text: '{translated_text}'\n"
    "Self-Assessment: '{self_assessment}'\n"
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model= llama3_bB_8192,
        max_tokens=1000,
        # temperature=temperature  # Remove this line as 'temperature' is not defined in this function
    )
    return chat_completion.choices[0].message.content.strip()

In [99]:
def main():
    text = "Break a leg!"
    source_lang = "English"
    target_lang = "عربي"

    translated_text = translate_text(text,source_lang, target_lang) # Use target_lang instead of target_language
    print("Translated Text:", translated_text)

    self_assessment = self_assess_translation(text, translated_text, target_lang)
    print("Self-Assessment:", self_assessment)

    resolved_translation = resolve_self_assessment(text, translated_text, self_assessment)
    print("Resolved Translation:", resolved_translation)

main()

Translated Text: Here's the translation:

**Translation:** كلّ الحظّ السعيد! (Kul al-ḥaẓ al-saʿīd!)

**Explanation:** The idiomatic expression "Break a leg!" is a well-known English phrase used to wish someone good luck, especially before a performance or a challenging situation. In Arabic, the literal translation of "Break a leg!" doesn't convey the same meaning, so a more culturally equivalent expression is needed.

The chosen translation, كلّ الحظّ السعيد! (Kul al-ḥaẓ al-saʿīd!), is a common Arabic phrase that conveys a similar sentiment. It literally means "All the best luck!" or "Good luck in everything!" This phrase is widely used in Arabic-speaking cultures to express good wishes and support before an important event or undertaking.

In Arabic culture, it's common to use phrases that invoke blessings and good fortune, rather than literal expressions that might be perceived as negative or violent (like "Break a leg!"). This translation choice ensures that the cultural nuance and 